# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor, BaggingClassifier, RandomForestClassifier,AdaBoostClassifier, GradientBoostingClassifier

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [3]:
#your code here
spaceship_cleaned = spaceship.dropna()
spaceship_cleaned['Cabin'] = spaceship_cleaned['Cabin'].str[0]
spaceship_cleaned.drop(columns=['PassengerId', 'Name'], inplace=True)
categorical_cols = ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP']
spaceship_cleaned_encoded = pd.get_dummies(spaceship_cleaned, columns=categorical_cols, drop_first=True)

#your code here

features = spaceship_cleaned_encoded.drop(columns = ["Transported"])
target = spaceship_cleaned_encoded["Transported"]
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state=0)

from sklearn.preprocessing import MinMaxScaler, StandardScaler
normalizer = MinMaxScaler()

normalizer.fit(X_train)

X_train_norm = normalizer.transform(X_train)

X_test_norm = normalizer.transform(X_test)

X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns)
X_test_norm = pd.DataFrame(X_test_norm, columns = X_test.columns)

C:\Users\tevfi\AppData\Local\Temp\ipykernel_6512\2246876157.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spaceship_cleaned['Cabin'] = spaceship_cleaned['Cabin'].str[0]
C:\Users\tevfi\AppData\Local\Temp\ipykernel_6512\2246876157.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spaceship_cleaned.drop(columns=['PassengerId', 'Name'], inplace=True)


- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [4]:
#your code here
forest_class = RandomForestClassifier(n_estimators=100,
                             max_depth=10, random_state=42)

forest_class.fit(X_train_norm, y_train)


RandomForestClassifier(max_depth=10, random_state=42)

- Evaluate your model

In [5]:

forest_class.score(X_test_norm, y_test)

0.7881996974281392

**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [18]:
#your code here
from sklearn.model_selection import GridSearchCV

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 200],  # Number of trees in the forest
    'max_leaf_nodes': [None, 10, 20, 50],  # Maximum number of leaf nodes
    'max_depth': [None, 10, 20, 30]  # Maximum depth of the tree
}


- Run Grid Search

In [ ]:
# Initialize the Random Forest Classifier
forest_class = RandomForestClassifier(random_state=42)

In [19]:
# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=forest_class,  # The model to tune
    param_grid=param_grid,   # The hyperparameter grid
    cv=5,                   # Number of cross-validation folds
    scoring='accuracy',     # Metric to optimize
    n_jobs=-1,             # Use all available CPU cores
    verbose=2              # Print progress
)

In [20]:
# Fit GridSearchCV to the training data
grid_search.fit(X_train_norm, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20, 30],
                         'max_leaf_nodes': [None, 10, 20, 50],
                         'n_estimators': [50, 100, 200]},
             scoring='accuracy', verbose=2)

- Evaluate your model

In [21]:
# Get the best hyperparameters
best_params = grid_search.best_params_
print(f"Best Hyperparameters: {best_params}")

Best Hyperparameters: {'max_depth': 10, 'max_leaf_nodes': None, 'n_estimators': 100}


In [22]:
# Evaluate the best model on the test data
best_model = grid_search.best_estimator_

In [23]:
test_accuracy = best_model.score(X_test_norm, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.7882
